In [1]:
import numpy as np
import matplotlib.pyplot as plt
import limetr
import pandas as pd
import time
import os


In [3]:
# first testing out limetr functionality
# later looping through replicates for the paper

df = pd.read_csv("/home/j/temp/reed/prog/projects/metareg_simulations/replicates/data/df2_0.csv")
df = df.sort_values(by=['study_id'])

# num_studies = 5
# study_sizes = [10]*num_studies
# num_obs = sum(study_sizes)
# k_beta = 2
# k_gamma = 1
# k_delta = num_studies
# beta_t = np.random.randn(k_beta)
# gamma_t = 0.01
# delta_t = np.random.rand(k_delta)*0.05
# X = np.vstack((np.ones(num_obs), np.linspace(0.0, 2.0, num_obs))).T
# Z = np.ones((num_obs, 1))
# z = np.split(Z, np.cumsum(study_sizes)[:-1])
# u = np.random.randn(num_studies, k_gamma)*np.sqrt(gamma_t)
# U = np.hstack([z[i].dot(u[i]) for i in range(num_studies)])
# E = np.random.randn(num_obs)*np.sqrt(np.repeat(delta_t, study_sizes))
# obs_mean = X.dot(beta_t) + U + E

# # create covariates for beta
# # function
# F = lambda beta: X.dot(beta)
# # Jacobian of the function
# JF = lambda beta: X

num_studies = np.unique(df['study_id'].values).size
study_size = int(df.shape[0] / num_studies)
study_sizes = [study_size]*num_studies

num_obs = sum(study_sizes)
k_beta = 2
k_gamma = 1
# k_delta defined in section farther below
beta_t = np.array([0, 5])
gamma_t = 6
X = np.vstack((np.ones(num_obs), df['x1'])).T
Z = np.ones((num_obs, 1))
z = np.split(Z, np.cumsum(study_sizes)[:-1])
obs_mean = df['y'].values
F = lambda beta: X.dot(beta)
JF = lambda beta: X
y_se = df['y_se'].values


##### case 1

k_delta = 20
uprior = np.array([
    [-np.inf]*k_beta + [1e-7]*k_gamma,
    [np.inf]*k_beta + [np.inf]*k_gamma
])

lt = limetr.LimeTr(
    study_sizes, k_beta, k_gamma, obs_mean, F, JF, Z,
    inlier_percentage=0.8, uprior=uprior, S = y_se
)

start = time.time()
beta_soln, gamma_soln, w_soln = lt.fitModel(outer_max_iter=100000, 
                                            outer_step_size=200.0)
end = time.time()
print(end - start)

delta_soln = lt.delta

print('true beta', beta_t)
print('soln beta', beta_soln)

print('true gamma', gamma_t)
print('soln gamma', np.sqrt(gamma_soln))

print('soln delta', np.sqrt(delta_soln))

# count number of true outliers detected
sum((lt.w == 0) & (df['outlier_status'] == 1))

0.7168939113616943
true beta [0 5]
soln beta [3.23619408 4.60529666]
true gamma 6
soln gamma [6.36337685]
soln delta []


15

In [4]:
##### case 2, k_delta = 1
k_delta = 1
uprior = np.array([
    [-np.inf]*k_beta + [1e-7]*k_gamma + [1e-7]*k_delta,
    [np.inf]*k_beta + [np.inf]*k_gamma + [100.0]*k_delta
])

lt = limetr.LimeTr(
    study_sizes, k_beta, k_gamma, obs_mean, F, JF, Z,
    inlier_percentage=0.8, uprior=uprior, share_obs_std=True
    # inlier_percentage=0.8, uprior=uprior
)

start = time.time()
beta_soln, gamma_soln, w_soln = lt.fitModel(outer_max_iter=100000,
                                            outer_step_size=5.0)
lt.optimize(x0=lt.soln, print_level=5, max_iter=10000)
end = time.time()
print(end - start)

delta_soln = lt.delta

print('true beta', beta_t)
print('soln beta', beta_soln)

print('true gamma', gamma_t)
print('soln gamma', np.sqrt(gamma_soln))

print('soln delta', np.sqrt(delta_soln))

# count number of true outliers detected
sum((lt.w == 0) & (df['outlier_status'] == 1))

0.9690077304840088
true beta [0 5]
soln beta [3.27399537 4.59596615]
true gamma 6
soln gamma [6.42032529]
soln delta [3.39088626]


15

In [20]:
##### case 2, k_delta = 20
k_delta = num_studies
uprior = np.array([
    [-np.inf]*k_beta + [1e-7]*k_gamma + [1e-7]*k_delta,
    [np.inf]*k_beta + [np.inf]*k_gamma + [100.0]*k_delta
])

lt = limetr.LimeTr(
    study_sizes, k_beta, k_gamma, obs_mean, F, JF, Z,
    # inlier_percentage=0.8, uprior=uprior, share_obs_std=True
    inlier_percentage=0.8, uprior=uprior
)

start = time.time()
beta_soln, gamma_soln, w_soln = lt.fitModel(outer_max_iter=100000,
                                            outer_step_size=5.0)
# lt.optimize(x0=lt.soln, print_level=5, max_iter=10000)
end = time.time()
print(end - start)

delta_soln = lt.delta

print('true beta', beta_t)
print('soln beta', beta_soln)

print('true gamma', gamma_t)
print('soln gamma', np.sqrt(gamma_soln))

print('soln delta', np.sqrt(delta_soln))

# count number of true outliers detected
sum((lt.w == 0) & (df['outlier_status'] == 1))

0.6675703525543213
true beta [0 5]
soln beta [3.30329634 4.61607681]
true gamma 6
soln gamma [6.54975808]
soln delta [2.93727313 2.34677749 2.63419785 5.12353417 5.13648676 0.78800072
 3.38606425 3.06436357 2.89289632 3.31318591]


15

In [5]:
##### case 2, k_delta = 1, loop through replicates
# -- longitudinal analysis


replicate_data_dir = "/home/j/temp/reed/prog/projects/metareg_simulations/replicates/"

for replicate_id in range(0, 30):

    # replicate_id = 22 # dev

    df_i = pd.read_csv(replicate_data_dir + "data/df2_" + str(replicate_id) + ".csv")
    df_i = df_i.sort_values(by=['study_id'])

    n_outliers = 15
    n_outlier_assumed = 20

    num_studies = np.unique(df_i['study_id'].values).size
    study_size = int(df_i.shape[0] / num_studies)
    study_sizes = [study_size]*num_studies
    num_obs = sum(study_sizes)
    assert num_obs == df_i.shape[0], "Check study sizes and/or input data"
    k_beta = 2
    k_gamma = 1
    beta_t = np.array([0, 5])
    gamma_t = 6
    delta_t = 4
    X = np.vstack((np.ones(num_obs), df_i['x1'])).T
    Z = np.ones((num_obs, 1))
    z = np.split(Z, np.cumsum(study_sizes)[:-1])
    obs_mean = df_i['y'].values
    F = lambda beta: X.dot(beta)
    JF = lambda beta: X

    k_delta = 1
    uprior = np.array([
        [-np.inf]*k_beta + [1e-7]*k_gamma + [1e-7]*k_delta,
        [np.inf]*k_beta + [np.inf]*k_gamma + [100.0]*k_delta
    ])

    lt = limetr.LimeTr(
        study_sizes, k_beta, k_gamma, obs_mean, F, JF, Z,
        inlier_percentage=0.8, uprior=uprior, share_obs_std=True
        # inlier_percentage=0.8, uprior=uprior
    )

    start = time.time()
    beta_soln, gamma_soln, w_soln = lt.fitModel(outer_max_iter=100000,
                                                outer_step_size=5.0)
    # lt.optimize(x0=lt.soln, print_level=5, max_iter=10000)
    end = time.time()
    runtime = end - start

    delta_soln = lt.delta
    tp = sum((lt.w == 0) & (df_i['outlier_status'] == 1))
    fp = sum((lt.w == 0) & (df_i['outlier_status'] == 0))
    tn = sum((lt.w == 1) & (df_i['outlier_status'] == 0))
    fn = sum((lt.w == 1) & (df_i['outlier_status'] == 1))

    df_out = pd.DataFrame({
        'model': ["LimeTR"],
        'beta0': beta_soln[0],
        'beta1': beta_soln[1],
        'gamma': np.sqrt(gamma_soln),
        'sigma': np.sqrt(delta_soln),
        'seconds': runtime,
        'true_outliers_detected': sum((lt.w == 0) & (df_i['outlier_status'] == 1)),
        'sample_size': df_i.shape[0]
    }).assign(
        re_beta0 = lambda x: x.beta0 - beta_t[0],
        re_beta1 = lambda x: x.beta1 - beta_t[1],
        re_gamma = lambda x: x.gamma - gamma_t,
        re_sigma = lambda x: x.sigma - delta_t,
        tpf = tp / n_outliers,
        fp_num = fp,
        fpf = lambda x: fp / (num_obs - n_outliers)
    )

    df_out.to_csv(replicate_data_dir + "results_case2_limetr/results_case2_limetr_" + str(replicate_id) + ".csv")

In [3]:
##### case 1, loop through replicates
# -- meta-analysis


replicate_data_dir = "/home/j/temp/reed/prog/projects/metareg_simulations/replicates/"

for replicate_id in range(0, 30):

    # replicate_id = 22 # dev

    df_i = pd.read_csv(replicate_data_dir + "data/df2_" + str(replicate_id) + ".csv")
    df_i = df_i.sort_values(by=['study_id'])

    n_outliers = 15
    n_outlier_assumed = 20

    num_studies = np.unique(df_i['study_id'].values).size
    study_size = int(df_i.shape[0] / num_studies)
    study_sizes = [study_size]*num_studies
    num_obs = sum(study_sizes)

    assert num_obs == df_i.shape[0], "Check study sizes and/or input data"
    k_beta = 2
    k_gamma = 1
    beta_t = np.array([0, 5])
    gamma_t = 6
    X = np.vstack((np.ones(num_obs), df_i['x1'])).T
    Z = np.ones((num_obs, 1))
    z = np.split(Z, np.cumsum(study_sizes)[:-1])
    obs_mean = df_i['y'].values
    F = lambda beta: X.dot(beta)
    JF = lambda beta: X
    y_se = df['y_se'].values

    ###
    
    uprior = np.array([
        [-np.inf]*k_beta + [1e-7]*k_gamma,
        [np.inf]*k_beta + [np.inf]*k_gamma
    ])

    lt = limetr.LimeTr(
        study_sizes, k_beta, k_gamma, obs_mean, F, JF, Z,
        inlier_percentage=0.8, uprior=uprior, S = y_se
    )

    start = time.time()
    beta_soln, gamma_soln, w_soln = lt.fitModel(outer_max_iter=100000, 
                                                outer_step_size=200.0)
    end = time.time()
    runtime = end - start

    tp = sum((lt.w == 0) & (df_i['outlier_status'] == 1))
    fp = sum((lt.w == 0) & (df_i['outlier_status'] == 0))
    tn = sum((lt.w == 1) & (df_i['outlier_status'] == 0))
    fn = sum((lt.w == 1) & (df_i['outlier_status'] == 1))

    df_out = pd.DataFrame({
        'model': ["LimeTR"],
        'beta0': beta_soln[0],
        'beta1': beta_soln[1],
        'gamma': np.sqrt(gamma_soln),
        'seconds': runtime,
        'true_outliers_detected': sum((lt.w == 0) & (df_i['outlier_status'] == 1)),
        'sample_size': df_i.shape[0]
    }).assign(
        re_beta0 = lambda x: x.beta0 - beta_t[0],
        re_beta1 = lambda x: x.beta1 - beta_t[1],
        re_gamma = lambda x: x.gamma - gamma_t,
        tpf = tp / n_outliers,
        fp_num = fp,
        fpf = lambda x: fp / (num_obs - n_outliers)
    )

    df_out.to_csv(replicate_data_dir + "results_case1_limetr/results_case1_limetr_" + str(replicate_id) + ".csv")